# Simple example
Please find below a simple example of an inverted pendulum. The goal is, by using only 
side pushes, to bring the wand upward. 

The ocp to solve is as follow:
- Movement of $1s$ separted into $50$ shooting points
- States variables are the generalized position (q) and velocities (qdot)
- Side translation is bounded to $\pm10\ m$, and rotation to $\pm2\pi\ rad$. Velocities are bounded to $\pm10\ m/s$ and $\pm31.4\ rad/s$, respectively
- Control variables are the generalized forces (tau) 
- Side translation force is bounded to $\pm100\ N$ and the rotation force is not allowed (bonded to $0\ Nm$)
- Cost functions: minimize the side pushes forces at all time
- Initial guess is arbitrarily set to $0$ for all the variables

![SegmentLocal](img/slider.gif)

In [ ]:
import numpy as np
import biorbd_casadi as biorbd
from bioptim import *

In [ ]:
model = biorbd.Model("models/Slider1Leg.bioMod"), biorbd.Model("models/Slider1Leg.bioMod")
nq = model[0].nbQ()
ntau = model[0].nbGeneralizedTorque()

# This is your turn
n_shooting = 40, 40
final_time = 0.2, 0.4
tau_min, tau_max, tau_init = -200, 200, 0

dynamics = DynamicsList()
dynamics.add(DynamicsFcn.TORQUE_DRIVEN, with_contact=True)
dynamics.add(DynamicsFcn.TORQUE_DRIVEN)

objective_functions = ObjectiveList()
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key='tau', weight=0.1, phase=0)
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key='tau', weight=0.1, phase=1)

x_bounds = BoundsList()
x_bounds.add(bounds=QAndQDotBounds(model[0]))
x_bounds.add(bounds=QAndQDotBounds(model[1]))

x_bounds[0][:3, 0] = [0, 3*np.pi / 8, -3*np.pi / 4]
x_bounds[0][3:, 0] = 0
x_bounds[0].min[0, -1] = 0
x_bounds[0].max[0, -1] = 0.25 

x_bounds[1][0, -1] = 1
x_bounds[1][3, -1] = 0

u_bounds = BoundsList()
u_bounds.add([tau_min] * nq,
             [tau_max] * ntau)
u_bounds.add([tau_min] * ntau,
             [tau_max] * ntau)

u_bounds[0][0, :] = 0
u_bounds[1][0, :] = 0

x_init = InitialGuessList()
x_init.add([0, 3*np.pi / 8, -3*np.pi / 4] + [0] * nq)  
x_init.add([0, 0, 0] + [0] * nq)  

u_init = InitialGuessList()
u_init.add([tau_init] * ntau)
u_init.add([tau_init] * ntau)

# Constraints
constraints = ConstraintList()
constraints.add(
    ConstraintFcn.TRACK_CONTACT_FORCES,
    min_bound=0,
    max_bound=np.inf,
    node=Node.ALL,
    contact_index=0,  # z axis
)

ocp =  OptimalControlProgram(
    model,
    dynamics,
    n_shooting,
    final_time,
    x_init,
    u_init,
    x_bounds,
    u_bounds,
    objective_functions=objective_functions,
    constraints=constraints,
    n_threads=8,
)



In [ ]:
sol = ocp.solve(show_online_optim=False)


In [ ]:
sol.graphs(automatically_organize=False)